# 八股

## 1. 一条SQL查询语句是如何执行的？

1. 连接器: 连接器负责和客户端(数据库)进行创建连接、获取权限、维持和管理链接.
2. 缓存查取: 当 MySQL 接收到一个查询请求时, 首先检查查询是否在查询缓存中, 如果存在, 可能是以key-value的形式存放的. 如果命中, 立即返回结果.
3. 分析器: 分析器对 SQL 查询进行语法解析和语义分析. 分析器会根据查询的 SQL 语句, 确定表的读取顺序, 确定表的连接类型, 确定表的查询条件等.
4. 优化器: 优化器会根据 MySQL 版本, 查询的 SQL 语句, 表的结构等, 选择最好的执行计划. 例如, 可能的查询方式, 可能的 join 顺序等.
5. 执行器: 分析器只要要做什么, 优化器知道要怎么做, 最后就是交给执行器, 执行语句.

## 2. 事务的四大特性有哪些?

1. 原子性: 一个事务的处理, 只有全部完成和不完成则回滚, 不存在部分完成情况.
2. 一致性: 在执行事务的前后, 数据库从前一个一致性状态转换成新的一致性状态.
3. 隔离性: 多个事务的执行是互不相干的
4. 持久性: 一个事务一旦提交, 就是永久存储, 即使数据库崩溃也不会丢失.

## 3. 数据库的事务隔离级别有哪些?

### 1. 读未提交（Read Uncommitted）
想象有两个用户，分别操作一个银行账户。

- 事务A：开始转账操作，从账户中扣除100元。
- 事务B：在事务A还未提交时，读取了账户余额。

假如原余额是500元，事务A正在扣除100元，尚未提交，所以账户余额显示为400元。但实际上，这100元的扣款并未确定，事务A可能会回滚，余额仍应为500元。事务B读取到了事务A尚未提交的数据，这种情况叫“脏读”。

### 2. 读已提交（Read Committed）
在这种隔离级别下，事务只能读取已经提交的数据。

- 事务A：开始转账操作，扣除100元，并在操作完成后提交。
- 事务B：只有在事务A提交之后，才读取到账户余额的变化。

假设事务A成功扣除100元并提交，账户余额变为400元，事务B这时读取余额就是400元，不会读取到未提交的数据。因此，避免了“脏读”，但如果事务B在事务A提交之前读取了一次余额（500元），然后事务A提交后再读取一次余额（400元），就会发现数据不一致，这种情况叫“不可重复读”。

### 3. 可重复读（Repeatable Read）
在“可重复读”隔离级别下，事务期间多次读取同一数据会得到一致的结果。

- 事务A：开始转账操作，扣除100元，但在事务A提交前，事务B多次读取余额。
- 事务B：在事务A的操作期间，读取余额，无论多少次都只能看到事务开始时的余额（500元）。

![image.png](./images/image_可重复读.png)

因此，事务B在事务A提交前，多次读取都保持一致，解决了“不可重复读”的问题。但如果事务A在过程中新增了一条数据，比如新增一笔100元的存款，则事务B读取余额列表时可能看到新增的数据，这种现象叫“幻读”。

### 4. 序列化（Serializable）
在最高的隔离级别下，事务是完全串行化的，彼此之间没有并发操作。

- 事务A：转账扣除100元。
- 事务B：在事务A执行完并提交前，事务B无法执行任何读取操作。

这样，不会出现“脏读”、“不可重复读”或“幻读”，因为事务B只有在事务A完全执行完并提交后才能读取数据。